In [ ]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from scipy.interpolate import griddata

In [ ]:
# Load data from the .mat file
data = sio.loadmat('../data/ML_Optimal_method.mat')
eddy_double_LES = data['eddy_double_LES']
eddy_double_TBNN = data['eddy_double_TBNN']
Sij = data['Sij']
Rij = data['Rij']
x = data['x']
y = data['y']
x_point = data['x_point']
y_point = data['y_point']
bij_LES = data['bij_LES']
bij_TBNN = data['bij_TBNN']

### edd viscosity = function(Sij, Rij)

In [ ]:
# Prepare features and target variables
# Use columns 0, 1, 3, and 4 from Sij an Rij as features
X = np.concatenate((Sij[:, [0, 1, 3, 4]], Rij[:, [0, 1, 3, 4]]), axis=1)
Y = eddy_double_TBNN.ravel()
Y_LES = eddy_double_LES.ravel()

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define a candidate nonlinear function to fit the data
# The function is defined as:
#   y = a + b*sin(c*X1) + d*cos(e*X2) + f*exp(g*X3) + h*log(|i*X4| + 1)
def candidate_func(X_tuple, a, b, c, d, e, f, g, h, i):
    # Unpack the independent variables from the tuple
    X1, X2, X3, X4 = X_tuple
    return a + b * np.sin(c * X1) + d * np.cos(e * X2) + f * np.exp(g * X3) + h * np.log(np.abs(i * X4) + 1)

# Prepare the independent variables as a tuple
X1 = X_scaled[:, 0]
X2 = X_scaled[:, 1]
X3 = X_scaled[:, 2]
X4 = X_scaled[:, 3]
X_tuple = (X1, X2, X3, X4)

# Set an initial guess for the parameters
initial_guess = [1, 1, 1, 1, 1, 1, 1, 1, 1]

# Use curve_fit to fit the candidate nonlinear function to the data
popt, pcov = curve_fit(candidate_func, X_tuple, Y, p0=initial_guess, maxfev=10000)
print("Optimal parameters:", popt)

# Compute predictions using the fitted nonlinear model
Y_pred_nonlinear = candidate_func(X_tuple, *popt)

# Evaluate the nonlinear model performance
rmse_nonlinear = np.sqrt(mean_squared_error(Y, Y_pred_nonlinear))
r2_nonlinear = r2_score(Y, Y_pred_nonlinear)
print("Nonlinear model RMSE:", rmse_nonlinear)
print("Nonlinear model R2:", r2_nonlinear)

# Plot predictions vs. actual values for the nonlinear model
plt.figure(figsize=(10, 6))
plt.plot(Y, label='Actual eddy_double_TBNN', color='blue')
plt.plot(Y_pred_nonlinear, label='Nonlinear Model Prediction', color='red', linestyle='--')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.title('Nonlinear Model Prediction vs. Actual')
plt.legend()
plt.grid(True)
plt.show()

# Interpolate the results onto the spatial grid
# This block uses griddata to map the original and predicted values to the spatial grid defined by x and y.
points = np.column_stack((x_point.ravel(), y_point.ravel()))
eddy_double_LES_interp = griddata(points, eddy_double_LES.ravel(), (x, y), method='cubic').reshape(x.shape)
eddy_double_TBNN_interp = griddata(points, eddy_double_TBNN.ravel(), (x, y), method='cubic').reshape(x.shape)
Y_pred_interp = griddata(points, Y_pred_nonlinear.ravel(), (x, y), method='cubic').reshape(x.shape)

# Set common color scale limits for the spatial plots
vmin = -0.05
vmax = 0.20

# Plot the interpolated maps in a vertical stack (3 subplots)
fig, axs = plt.subplots(3, 1, figsize=(8, 8))
im0 = axs[0].pcolormesh(x, y, eddy_double_LES_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[0].set_title('eddy double LES')
axs[0].set_aspect('equal', adjustable='box')
axs[0].set_xlim(np.min(x), np.max(x))
axs[0].set_ylim(np.min(y), np.max(y))

im1 = axs[1].pcolormesh(x, y, eddy_double_TBNN_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[1].set_title('eddy double TBNN')
axs[1].set_aspect('equal', adjustable='box')
axs[1].set_xlim(np.min(x), np.max(x))
axs[1].set_ylim(np.min(y), np.max(y))

im2 = axs[2].pcolormesh(x, y, Y_pred_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[2].set_title('Nonlinear Model Y pred')
axs[2].set_aspect('equal', adjustable='box')
axs[2].set_xlim(np.min(x), np.max(x))
axs[2].set_ylim(np.min(y), np.max(y))

fig.subplots_adjust(right=0.85, hspace=0.1)
cbar_ax = fig.add_axes([0.88, 0.15, 0.02, 0.7])
fig.colorbar(im2, cax=cbar_ax)
plt.show()

### edd viscosity = function(Sij)

In [ ]:
# Prepare features and target variable
# Use columns 0, 1, 3, and 4 from Sij as features
X = Sij[:, [0, 1, 3, 4]]
Y = eddy_double_TBNN.ravel()
Y_LES = eddy_double_LES.ravel()

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define a candidate nonlinear function to fit the data
# The function is defined as:
#   y = a + b*sin(c*X1) + d*cos(e*X2) + f*exp(g*X3) + h*log(|i*X4| + 1)
def candidate_func(X_tuple, a, b, c, d, e, f, g, h, i):
    # Unpack the independent variables from the tuple
    X1, X2, X3, X4 = X_tuple
    return a + b * np.sin(c * X1) * d * np.cos(e * X2) + f * np.exp(g * X3) + h * np.log(np.abs(i * X4) + 1)

# Prepare the independent variables as a tuple
X1 = X_scaled[:, 0]
X2 = X_scaled[:, 1]
X3 = X_scaled[:, 2]
X4 = X_scaled[:, 3]
X_tuple = (X1, X2, X3, X4)

# Set an initial guess for the parameters
initial_guess = [1, 1, 1, 1, 1, 1, 1, 1, 1]

# Use curve_fit to fit the candidate nonlinear function to the data
popt, pcov = curve_fit(candidate_func, X_tuple, Y, p0=initial_guess, maxfev=10000)
print("Optimal parameters:", popt)

# Compute predictions using the fitted nonlinear model
Y_pred_nonlinear = candidate_func(X_tuple, *popt)

# Evaluate the nonlinear model performance
rmse_nonlinear = np.sqrt(mean_squared_error(Y, Y_pred_nonlinear))
r2_nonlinear = r2_score(Y, Y_pred_nonlinear)
print("Nonlinear model RMSE:", rmse_nonlinear)
print("Nonlinear model R2:", r2_nonlinear)

# Plot predictions vs. actual values for the nonlinear model
plt.figure(figsize=(10, 6))
plt.plot(Y, label='Actual eddy_double_TBNN', color='blue')
plt.plot(Y_pred_nonlinear, label='Nonlinear Model Prediction', color='red', linestyle='--')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.title('Nonlinear Model Prediction vs. Actual')
plt.legend()
plt.grid(True)
plt.show()

# Interpolate the results onto the spatial grid
# This block uses griddata to map the original and predicted values to the spatial grid defined by x and y.
points = np.column_stack((x_point.ravel(), y_point.ravel()))
eddy_double_LES_interp = griddata(points, eddy_double_LES.ravel(), (x, y), method='cubic').reshape(x.shape)
eddy_double_TBNN_interp = griddata(points, eddy_double_TBNN.ravel(), (x, y), method='cubic').reshape(x.shape)
Y_pred_interp = griddata(points, Y_pred_nonlinear.ravel(), (x, y), method='cubic').reshape(x.shape)

# Set common color scale limits for the spatial plots
vmin = -0.05
vmax = 0.20

# Plot the interpolated maps in a vertical stack (3 subplots)
fig, axs = plt.subplots(3, 1, figsize=(8, 8))
im0 = axs[0].pcolormesh(x, y, eddy_double_LES_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[0].set_title('eddy double LES')
axs[0].set_aspect('equal', adjustable='box')
axs[0].set_xlim(np.min(x), np.max(x))
axs[0].set_ylim(np.min(y), np.max(y))

im1 = axs[1].pcolormesh(x, y, eddy_double_TBNN_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[1].set_title('eddy double TBNN')
axs[1].set_aspect('equal', adjustable='box')
axs[1].set_xlim(np.min(x), np.max(x))
axs[1].set_ylim(np.min(y), np.max(y))

im2 = axs[2].pcolormesh(x, y, Y_pred_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[2].set_title('Nonlinear Model Y pred')
axs[2].set_aspect('equal', adjustable='box')
axs[2].set_xlim(np.min(x), np.max(x))
axs[2].set_ylim(np.min(y), np.max(y))

fig.subplots_adjust(right=0.85, hspace=0.1)
cbar_ax = fig.add_axes([0.88, 0.15, 0.02, 0.7])
fig.colorbar(im2, cax=cbar_ax)
plt.show()

### edd viscosity = function(Sij, bij)

In [ ]:
# Prepare features and target variables
# Use columns 0, 1, 3, and 4 from Sij as features and bij_LES as additional features
X = np.concatenate((Sij[:, [0, 1, 3, 4]], bij_TBNN[:, [0, 1, 3, 4]]), axis=1)
Y = eddy_double_TBNN.ravel()
Y_LES = eddy_double_LES.ravel()

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define a candidate nonlinear function to fit the data
# The function is defined as:
#   y = a + b*sin(c*X1) + d*cos(e*X2) + f*exp(g*X3) + h*log(|i*X4| + 1)
def candidate_func(X_tuple, a, b, c, d, e, f, g, h, i):
    # Unpack the independent variables from the tuple
    X1, X2, X3, X4 = X_tuple
    return a + b * np.sin(c * X1) + d * np.cos(e * X2) + f * np.exp(g * X3) + h * np.log(np.abs(i * X4) + 1)

# Prepare the independent variables as a tuple
X1 = X_scaled[:, 0]
X2 = X_scaled[:, 1]
X3 = X_scaled[:, 2]
X4 = X_scaled[:, 3]
X_tuple = (X1, X2, X3, X4)

# Set an initial guess for the parameters
initial_guess = [1, 1, 1, 1, 1, 1, 1, 1, 1]

# Use curve_fit to fit the candidate nonlinear function to the data
popt, pcov = curve_fit(candidate_func, X_tuple, Y, p0=initial_guess, maxfev=10000)
print("Optimal parameters:", popt)

# Compute predictions using the fitted nonlinear model
Y_pred_nonlinear = candidate_func(X_tuple, *popt)

# Evaluate the nonlinear model performance
rmse_nonlinear = np.sqrt(mean_squared_error(Y, Y_pred_nonlinear))
r2_nonlinear = r2_score(Y, Y_pred_nonlinear)
print("Nonlinear model RMSE:", rmse_nonlinear)
print("Nonlinear model R2:", r2_nonlinear)

# Plot predictions vs. actual values for the nonlinear model
plt.figure(figsize=(10, 6))
plt.plot(Y, label='Actual eddy_double_TBNN', color='blue')
plt.plot(Y_pred_nonlinear, label='Nonlinear Model Prediction', color='red', linestyle='--')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.title('Nonlinear Model Prediction vs. Actual')
plt.legend()
plt.grid(True)
plt.show()

# Interpolate the results onto the spatial grid
# This block uses griddata to map the original and predicted values to the spatial grid defined by x and y.
points = np.column_stack((x_point.ravel(), y_point.ravel()))
eddy_double_LES_interp = griddata(points, eddy_double_LES.ravel(), (x, y), method='cubic').reshape(x.shape)
eddy_double_TBNN_interp = griddata(points, eddy_double_TBNN.ravel(), (x, y), method='cubic').reshape(x.shape)
Y_pred_interp = griddata(points, Y_pred_nonlinear.ravel(), (x, y), method='cubic').reshape(x.shape)

# Set common color scale limits for the spatial plots
vmin = -0.05
vmax = 0.20

# Plot the interpolated maps in a vertical stack (3 subplots)
fig, axs = plt.subplots(3, 1, figsize=(8, 8))
im0 = axs[0].pcolormesh(x, y, eddy_double_LES_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[0].set_title('eddy double LES')
axs[0].set_aspect('equal', adjustable='box')
axs[0].set_xlim(np.min(x), np.max(x))
axs[0].set_ylim(np.min(y), np.max(y))

im1 = axs[1].pcolormesh(x, y, eddy_double_TBNN_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[1].set_title('eddy double TBNN')
axs[1].set_aspect('equal', adjustable='box')
axs[1].set_xlim(np.min(x), np.max(x))
axs[1].set_ylim(np.min(y), np.max(y))

im2 = axs[2].pcolormesh(x, y, Y_pred_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[2].set_title('Nonlinear Model Y pred')
axs[2].set_aspect('equal', adjustable='box')
axs[2].set_xlim(np.min(x), np.max(x))
axs[2].set_ylim(np.min(y), np.max(y))

fig.subplots_adjust(right=0.85, hspace=0.1)
cbar_ax = fig.add_axes([0.88, 0.15, 0.02, 0.7])
fig.colorbar(im2, cax=cbar_ax)
plt.show()